In [1]:
import pandas as pd
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as Data
import lmdb

from torch.autograd import Variable
from warpctc_pytorch import CTCLoss
import sys

from torch.utils.data import Dataset
import torchvision.transforms as transforms
import six
import sys
from PIL import Image

In [2]:
sys.path.insert(1,'./Test/')

In [3]:
import models.crnn as crnn
import dataset
import utils

In [4]:
batchSize = 64

In [5]:
outputPath ='/image/.Eric/lmdb_train'

In [6]:
train_dataset = dataset.lmdbDataset(root=outputPath)

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize,shuffle=True,num_workers=4,
                                           collate_fn=dataset.alignCollate(imgH=32, imgW=320))

In [8]:
alphabet = '零壹貳參肆伍陸柒捌玖拾佰仟萬億兆元整'

In [9]:
nclass = len(alphabet) + 1
nc = 1
criterion = CTCLoss()

In [10]:
converter = utils.strLabelConverter(alphabet)
criterion = CTCLoss()

In [11]:
model = crnn.CRNN(32, 1, 19, 256)
model = model.cuda()
model.load_state_dict(torch.load("./Test/data/crnnV1.pkl"))

In [12]:
image = torch.FloatTensor(batchSize, 1, 32, 32)
text = torch.IntTensor(batchSize * 5)
length = torch.IntTensor(batchSize)

In [13]:
model = torch.nn.DataParallel(model)
image = image.cuda()
criterion = criterion.cuda()

In [14]:
image = Variable(image)
text = Variable(text)
length = Variable(length)

In [15]:
# loss averager
loss_avg = utils.averager()

In [16]:
optimizer = optim.Adam(model.parameters(), lr=0.01126,betas=(0.01, 0.999))

In [40]:
def trainBatch(net, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    utils.loadData(image, cpu_images)
    t, l = converter.encode(cpu_texts)
    utils.loadData(text, t)
    utils.loadData(length, l)

    preds = model(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    model.zero_grad()
    cost.backward()
    optimizer.step()
    return cost

In [41]:
for epoch in range(10):
    train_iter = iter(train_loader)
    i = 0
    while i < len(train_loader):
        for p in model.parameters():
            p.requires_grad = True
        model.train()
        cost = trainBatch(model, criterion, optimizer)
        loss_avg.add(cost)
        
        i += 1

        if i % 10 == 0:
            print('[iteration%d/%d][%d/%d] Loss: %f' %
                  (epoch, 10, i, len(train_loader), loss_avg.val() ))
            loss_avg.reset()

[iteration0/10][10/66] Loss: 536.871826
[iteration0/10][20/66] Loss: 224.750885
[iteration0/10][30/66] Loss: 119.379799
[iteration0/10][40/66] Loss: 98.599136
[iteration0/10][50/66] Loss: 140.716614
[iteration0/10][60/66] Loss: 87.206322
[iteration1/10][10/66] Loss: 73.583549
[iteration1/10][20/66] Loss: 69.220596
[iteration1/10][30/66] Loss: 62.676178
[iteration1/10][40/66] Loss: 62.416626
[iteration1/10][50/66] Loss: 59.654400
[iteration1/10][60/66] Loss: 55.787224
[iteration2/10][10/66] Loss: 55.591343
[iteration2/10][20/66] Loss: 51.742989
[iteration2/10][30/66] Loss: 51.204010
[iteration2/10][40/66] Loss: 49.588894
[iteration2/10][50/66] Loss: 48.655857
[iteration2/10][60/66] Loss: 47.228687
[iteration3/10][10/66] Loss: 45.347446
[iteration3/10][20/66] Loss: 44.837746
[iteration3/10][30/66] Loss: 45.071877
[iteration3/10][40/66] Loss: 43.688950
[iteration3/10][50/66] Loss: 42.680870
[iteration3/10][60/66] Loss: 40.689514
[iteration4/10][10/66] Loss: 40.363163
[iteration4/10][20/66

In [44]:
torch.save(model.module.state_dict(), './Test/data/crnnV2.pkl')

In [52]:
converter.decode(output,len(output))

AttributeError: 'int' object has no attribute 'numel'

In [51]:
len(output)

81

In [50]:
output = [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  8,  5, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 17, 17, 18]

In [53]:
converter.dict          740元整

{'零': 1,
 '壹': 2,
 '貳': 3,
 '參': 4,
 '肆': 5,
 '伍': 6,
 '陸': 7,
 '柒': 8,
 '捌': 9,
 '玖': 10,
 '拾': 11,
 '佰': 12,
 '仟': 13,
 '萬': 14,
 '億': 15,
 '兆': 16,
 '元': 17,
 '整': 18}

In [ ]:
'零壹貳參肆伍陸柒捌玖拾佰仟萬億兆元整'